# 협업필터링(CF) 추천 - Bias from mean

In [1]:
import numpy as np
print(f'NumPy v{np.__version__}')

import pandas as pd
print(f'pandas v{pd.__version__}')

import sklearn
print(f'scikit-learn v{sklearn.__version__}')

# Only for specifying versions
import sys; print(f'Python v{sys.version}')

NumPy v1.25.0
pandas v1.5.3
scikit-learn v1.2.0
Python v3.9.16 (main, May 17 2023, 17:49:16) [MSC v.1916 64 bit (AMD64)]


## 3.6 사용자의 평가경향을 고려한 CF

CF의 정확도를 더 개선시키는 방법 중의 하나는 사용자의 평가경향(user bias)을 고려해서 예측치를 조정하는 것이다. 사용자에 따라서 평가를 전체적으로 높게 하는 사람이 있는 반면에 평가를 전체적으로 낮게 하는 사람이 있는 등, 사람에 따라 평가경향이 다르다.

1. 각 사용자의 평점평균을 구한다.
2. 각 아이템의 평점을 각 사용자의 평균에서의 차이(평점 - 해당 사용자의 평점 평균)로 변환한다. 편의상 평점과 평균의 차이를 평점편차로 부르기로 한다.
3. 평점편차를 사용해서 해당 사용자의 해당 아이템의 편차 예측값(평점편차의 예측값)을 구한다. 구체적으로는 해당 사용자의 이웃을 구하고 이들 이웃의 해당 아이템에 대한 평점편차와 유사도를 가중평균한다.
4. 이렇게 구한 편차 예측값은 평균에서의 차이를 의미하기 때문에 실제 예측값으로 변환하기 위해서 현 사용자의 평균에 이 편차 예측값을 더해준다.
5. 예측값을 구할 수 없는 경우에 지금까지는 3.0을 할당했는데, 이번에는 해당 사용자의 평점평균으로 대체한다.

아래의 cell은 `3-2.ipynb`의 2~3번째 cell과 동일한 코드이다.

In [2]:
# 데이터 읽어 오기
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('../Data/u.user', sep='|', names=u_cols, encoding='latin-1')

i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown', 
          'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 
          'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
          'Thriller', 'War', 'Western']
movies = pd.read_csv('../Data/u.item', sep='|', names=i_cols, encoding='latin-1')

r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('../Data/u.data', sep='\t', names=r_cols, encoding='latin-1')

# movie ID와 title 빼고 다른 데이터 제거
movies = movies[['movie_id', 'title']]

# timestamp 제거 
ratings.drop('timestamp', axis='columns', inplace=True)

# train, test set 분리
from sklearn.model_selection import train_test_split
x = ratings.copy()
y = ratings['user_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42, stratify=y)

# 정확도(RMSE)를 계산하는 함수
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

# 모델별 RMSE를 계산하는 함수 
def score(model, neighbor_size=0):
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    y_pred = np.array([model(user, movie, neighbor_size) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

# train 데이터로 Full matrix 구하기 
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')

# train set의 모든 가능한 사용자 pair의 Cosine similarities 계산
from sklearn.metrics.pairwise import cosine_similarity
u_id = rating_matrix.index
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=u_id, columns=u_id)

In [3]:
rating_matrix.shape

(943, 1641)

In [4]:
# train 데이터의 user의 rating 평균과 영화의 평점편차 계산
rating_mean = rating_matrix.mean(axis='columns')
rating_mean.shape

(943,)

In [5]:
rating_bias = (rating_matrix.T - rating_mean).T
rating_bias.shape

(943, 1641)

In [6]:
def CF_knn_bias(user_id, movie_id, neighbor_size=0):
    if movie_id in rating_bias:
        # 현 user와 다른 사용자 간의 유사도 가져오기
        sim_scores = user_similarity[user_id].copy()

        # 현 movie의 평점편차 가져오기
        movie_ratings = rating_bias[movie_id].copy()

        # 현 movie에 대한 rating이 없는 사용자 삭제
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        movie_ratings.drop(none_rating_idx, inplace=True)
        sim_scores.drop(none_rating_idx, inplace=True)

        if neighbor_size == 0:  # Neighbor size가 지정되지 않은 경우
            # 편차로 예측값(편차 예측값) 계산
            prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
            # 편차 예측값에 현 사용자의 평균 더하기
            prediction += rating_mean[user_id]

        else:                   # Neighbor size가 지정된 경우
            # 해당 영화를 평가한 사용자가 최소 2명이 되는 경우에만 계산            
            if len(sim_scores) > 1:
                # 지정된 neighbor size 값과 해당 영화를 평가한 총사용자 수 중 작은 것으로 결정
                neighbor_size = min(neighbor_size, len(sim_scores))

                # array로 바꾸기 (argsort를 사용하기 위함)
                sim_scores = sim_scores.to_numpy()
                movie_ratings = movie_ratings.to_numpy()

                # 유사도를 순서대로 정렬
                user_idx = np.argsort(sim_scores)
                selected_idx = user_idx[-neighbor_size:]

                # 유사도와 rating을 neighbor size만큼 받기
                sim_scores = sim_scores[selected_idx]
                movie_ratings = movie_ratings[selected_idx]

                # 편차로 예측치 계산
                prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
                # 예측값에 현 사용자의 평균 더하기
                prediction += rating_mean[user_id]
            else:
                prediction = rating_mean[user_id]
    else:
        prediction = rating_mean[user_id]

    # if prediction < 1:
    #     return 1
    # if prediction > 5:
    #     return 5
    return prediction

score(CF_knn_bias, neighbor_size=30)

0.9485429509551335

RMSE가 0.949로 사용자의 평가경향을 고려하지 않은 `3-2.ipynb`의 `cf_knn()`(1.017)보다 크게 개선되었음을 알 수 있다.

다만, 알고리즘의 특성 상 prediction이 1 미만 혹은 5 초과의 값을 가질 수 있는데 (아래 연습문제의 결과 참고), `return` 전에 post processing으로 clipping을 해주면 결과가 조금 더 정확할 수 있다. (clipping 후의 RMSE는 0.947)

## 연습문제

Q. 코드를 수정하여 사용자 ID를 지정하면 해당 사용자를 위해 5개의 영화를 추천하도록 하라. 그 결과를 3.4의 추천 결과(`3-2.ipynb`)와 비교하라.

In [7]:
movies.set_index('movie_id', inplace=True)

In [8]:
# 전체 데이터로 full matrix와 cosine similarity 구하기
rating_matrix = ratings.pivot_table(values='rating', index='user_id', columns='movie_id')
target_user = 2

u_id = rating_matrix.index
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=u_id, columns=u_id)
user_similarity.shape

def recommender(user, n_items=10, neighbor_size=20):
    # 현재 사용자의 모든 아이템에 대한 예상 평점 계산
    rated_index = rating_matrix.loc[user][rating_matrix.loc[user] > 0].index    # 이미 평가한 영화 확인
    items = rating_matrix.loc[user].drop(rated_index)
    i_id = items.index

    predictions = [CF_knn_bias(user, item, neighbor_size) for item in i_id]   # 예상평점 계산

    recommendations = pd.Series(data=predictions, index=i_id, dtype=float)
    recommendations = recommendations.nlargest(n_items)    # 예상평점이 가장 높은 영화 선택
    recommended_items = movies.loc[recommendations.index]['title']
    return pd.concat({'prediction': recommendations, 'movie title': recommended_items}, axis=1)

recommender(user=target_user, n_items=10, neighbor_size=30)

,prediction,movie title
movie_id,,
851,5.614679,Two or Three Things I Know About Her (1966)
1512,5.332438,"World of Apu, The (Apur Sansar) (1959)"
1467,5.196731,"Saint of Fort Washington, The (1993)"
1591,5.195337,Duoluo tianshi (1995)
1293,5.119964,Star Kid (1997)
1500,5.064014,Santa with Muscles (1996)
1449,5.007031,Pather Panchali (1955)
1499,4.930905,Grosse Fatigue (1994)
1398,4.926353,Anna (1996)


`user_id = 2`에 대해 상위 10개 추천 영화를 받아보면, `3-2.ipynb`와 겹치는 영화는 851, 1512, 1467, 1293, 1500, 1443의 6개인 것을 알 수 있다.